In [1]:
%load_ext autoreload
%autoreload 2

%pip install -r ../../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
flights = pd.read_parquet("../../resources/flights_clean.parquet")
flights.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Semana,Mes
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,42,10
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,27,7
2,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,26,6
3,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,6,2
4,3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,125.0,36,9


La variable real a predecir el modelo es la cantida de pasajeros de un vuelo individual

In [4]:
y_real = flights["Passengers"]
y_real.head()

0    229.0
1    197.0
2    200.0
3    162.0
4    183.0
Name: Passengers, dtype: float64

In [5]:
X = flights[["Capacity", "Bookings"]]
y = flights["Passengers"]

In [15]:
import xgboost as xg
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=123,
)

eval_set = [(test_x, test_y)]

train_dmatrix = xg.DMatrix(data=train_x, label=train_y)
test_dmatrix = xg.DMatrix(data=test_x, label=test_y)

# xgb_r = XGBRegressor(n_estimators=10, seed=123)
xgb_r = XGBRegressor()
xgb_r

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [16]:
xgb_r.fit(train_x, train_y, eval_set=eval_set, verbose=True)

[0]	validation_0-rmse:30.92216
[1]	validation_0-rmse:27.11982
[2]	validation_0-rmse:25.03549
[3]	validation_0-rmse:23.94214
[4]	validation_0-rmse:23.38936
[5]	validation_0-rmse:23.11311
[6]	validation_0-rmse:22.97551
[7]	validation_0-rmse:22.91210
[8]	validation_0-rmse:22.88275
[9]	validation_0-rmse:22.86872
[10]	validation_0-rmse:22.86178
[11]	validation_0-rmse:22.86164
[12]	validation_0-rmse:22.86204
[13]	validation_0-rmse:22.86121
[14]	validation_0-rmse:22.86528
[15]	validation_0-rmse:22.86644
[16]	validation_0-rmse:22.87091
[17]	validation_0-rmse:22.87179
[18]	validation_0-rmse:22.87138
[19]	validation_0-rmse:22.87428
[20]	validation_0-rmse:22.87557
[21]	validation_0-rmse:22.87479
[22]	validation_0-rmse:22.87394
[23]	validation_0-rmse:22.87670
[24]	validation_0-rmse:22.87867
[25]	validation_0-rmse:22.87987
[26]	validation_0-rmse:22.88086
[27]	validation_0-rmse:22.88189
[28]	validation_0-rmse:22.88456
[29]	validation_0-rmse:22.88514
[30]	validation_0-rmse:22.88702
[31]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [18]:
xgb_r.predict(test_x)

array([158.0215 , 172.8005 , 171.69553, ..., 187.14119, 149.38206,
       173.3334 ], dtype=float32)

In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
aeronave = le.fit_transform(flights["Aeronave"])
pd.Series(aeronave).value_counts()

69    2228
35    2198
37    2196
39    2142
80    2112
      ... 
1       90
41      82
0       78
2       32
4       31
Name: count, Length: 84, dtype: int64